# Thrust allocation tutorial

## Polar coordinate thrust allocation

The polar coordinate thrust allocation for a 3 DOF vehicle with $m$ thrusters is given by the following equation:
$$
B(\alpha) = \begin{bmatrix}
\cos(\alpha_1) & \cos(\alpha_2) & \cdots & \cos(\alpha_m) \\
\sin(\alpha_1) & \sin(\alpha_2) & \cdots & \sin(\alpha_m) \\
l_{x,1} \sin(\alpha_1) - l_{y,1} \cos(\alpha_1) & l_{x,2} \sin(\alpha_2) - l_{y,2} \cos(\alpha_2) & \cdots & l_{x,m} \sin(\alpha_m) - l_{y,m} \cos(\alpha_m)
\end{bmatrix}
$$
where $\alpha_i$ is the angle of the $i$-th thruster with respect to the x-axis, and $l_{x,i}$ and $l_{y,i}$ are the x and y coordinates of the $i$-th thruster.

For simplicity, you can consider a single column to represent a single thruster configuration, i.e.:
$$
B_i(\alpha_i) = \begin{bmatrix}
\cos(\alpha_i) \\
\sin(\alpha_i) \\
l_{x,i} \sin(\alpha_i) - l_{y,i} \cos(\alpha_i)
\end{bmatrix}
$$


In [1]:
# Create a ipynb slider to select an angle for port and starboard thrusters
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import ipywidgets
import scipy as sp
import scipy.linalg

a1 = 0
a2 = 0
@interact(
    port=widgets.FloatSlider(min=-np.pi, max=np.pi, step=0.01, value=0),
    starboard=widgets.FloatSlider(min=-np.pi, max=np.pi, step=0.01, value=0))
def f1(port, starboard):
    global a1, a2
    a1, a2 = port, starboard

w0 = 1.0
w1 = 1.0
w2 = 1.0
@interact(
    _w0=widgets.FloatSlider(min=0, max=10, step=0.01, value=1, description='w0'),
    _w1=widgets.FloatSlider(min=0, max=10, step=0.01, value=1, description='w1'),
    _w2=widgets.FloatSlider(min=0, max=10, step=0.01, value=1, description='w2'))
def f2(_w0, _w1, _w2):
    global w0, w1, w2
    w0, w1, w2 = _w0, _w1, _w2

l_x0 = 1.0
l_x1 = 0.0
l_x2 = 0.0

l_y0 = 0.0
l_y1 = -1.0
l_y2 = 1.0

interactive(children=(FloatSlider(value=0.0, description='port', max=3.141592653589793, min=-3.141592653589793…

interactive(children=(FloatSlider(value=1.0, description='w0', max=10.0, step=0.01), FloatSlider(value=1.0, de…

In [2]:
# Create a 3DOF thrust allocation matrix using the angles and single tunnel
# thruster. Save it on a matrix called B
B = np.array(
    [[0, np.cos(a1), np.cos(a2)],
     [1, np.sin(a1), np.sin(a2)],
     [l_x0, l_x1*np.sin(a1) - l_y1*np.cos(a2), l_x2*np.sin(a2) - l_y2*np.cos(a2)]])

# Create a 3x3 identity matrix for the weights
W = np.diag([w0, w1, w2])

Now write down $$f^* = B_W^\dagger \tau_{cmd} + Q_W f_d, \quad Q_W := I - B_W^\dagger B$$

Here, we use $Q_W$ to represents a correction term to pull the solution closer to the desired thrusts $f_d$. That can be useful and thought of as a way to minimize the norm of the thruster forces.

and recall that $$B_W^\dagger := W^{-1}B^\top [B W^{-1}B^\top]^{-1}.$$

First compute $B_W^\dagger$ and $Q_W$.

In [3]:
B_pinv_W = np.linalg.inv(W) @ B.T @ np.linalg.inv(B @ np.linalg.inv(W) @ B.T)

Q_W = np.identity(3) - B_pinv_W @ B

Now form the $\tau_{cmd}$ vector.

In [4]:
f_x, f_y, m_z = 0, 0, 0
tau_cmd = np.array([[f_x], [f_y], [m_z]])
@interact(
    Fx=widgets.FloatSlider(min=-100, max=100, step=0.01, value=0, description='Fx'),
    Fy=widgets.FloatSlider(min=-100, max=100, step=0.01, value=0, description='Fy'),
    Mz=widgets.FloatSlider(min=-100, max=100, step=0.01, value=0, description='Mz'))
def f3(Fx, Fy, Mz):
    global f_x, f_y, m_z, tau_cmd
    f_x = Fx
    f_y = Fy
    m_z = Mz
    tau_cmd = np.array([[f_x], [f_y], [m_z]])



interactive(children=(FloatSlider(value=0.0, description='Fx', min=-100.0, step=0.01), FloatSlider(value=0.0, …

Now compute $f^*$.

In [5]:
f_opt = B_pinv_W @ tau_cmd + Q_W @ np.array([[0], [0], [0]])

f_opt

array([[0.],
       [0.],
       [0.]])

Now incorporate the $f_d$ term.

In [6]:
f_d0, f_d1, f_d2 = 0, 0, 0
f_d = np.array([[f_d0], [f_d1], [f_d2]])
@interact(
    F0=widgets.FloatSlider(min=-100, max=100, step=0.01, value=0, description='Desired F0'),
    F1=widgets.FloatSlider(min=-100, max=100, step=0.01, value=0, description='Desired F1'),
    F2=widgets.FloatSlider(min=-100, max=100, step=0.01, value=0, description='Desired F2'))
def f3(F0, F1, F2):
    global f_d0, f_d1, f_d2, f_d
    f_d0 = F0
    f_d1 = F1
    f_d2 = F2
    f_d = np.array([[f_d0], [f_d1], [f_d2]])


interactive(children=(FloatSlider(value=0.0, description='Desired F0', min=-100.0, step=0.01), FloatSlider(val…

In [7]:
f_opt = B_pinv_W @ tau_cmd + Q_W @ f_d

print(f"f_opt = {f_opt}, \n and \n f_d = {f_d}")

f_opt = [[0.]
 [0.]
 [0.]], 
 and 
 f_d = [[0.]
 [0.]
 [0.]]
